# NLTK GUI Chatbot Train

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
#nltk.download('punkt')
#nltk.download('wordnet')
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


## Explore input data

In [2]:
import pandas as pd

train_json = pd.read_json('intents.json')

train_json
pd.set_option('display.max_colwidth', 300)
train_json.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}"
1,"{'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}"
2,"{'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', ""That's helpful"", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}"
3,"{'tag': 'noanswer', 'patterns': [], 'responses': [""Sorry, can't understand you"", 'Please give me more info', 'Not sure I understand'], 'context': ['']}"
4,"{'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies', 'Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies'], 'context': ['']}"
5,"{'tag': 'adverse_drug', 'patterns': ['How to check Adverse drug reaction?', 'Open adverse drugs module', 'Give me a list of drugs causing adverse behavior', 'List all drugs suitable for patient with adverse reaction', 'Which drugs dont have adverse reaction?'], 'responses': ['Navigating to Adverse drug reaction module'], 'context': ['']}"
6,"{'tag': 'blood_pressure', 'patterns': ['Open blood pressure module', 'Task related to blood pressure', 'Blood pressure data entry', 'I want to log blood pressure results', 'Blood pressure data management'], 'responses': ['Navigating to Blood Pressure module'], 'context': ['']}"
7,"{'tag': 'blood_pressure_search', 'patterns': ['I want to search for blood pressure result history', 'Blood pressure for patient', 'Load patient blood pressure result', 'Show blood pressure results for patient', 'Find blood pressure results by ID'], 'responses': ['Please provide Patient ID', 'Patient ID?'], 'context': ['search_blood_pressure_by_patient_id']}"
8,"{'tag': 'search_blood_pressure_by_patient_id', 'patterns': [], 'responses': ['Loading Blood pressure result for Patient'], 'context': ['']}"
9,"{'tag': 'pharmacy_search', 'patterns': ['Find me a pharmacy', 'Find pharmacy', 'List of pharmacies nearby', 'Locate pharmacy', 'Search pharmacy'], 'responses': ['Please provide pharmacy name'], 'context': ['search_pharmacy_by_name']}"


The json contains 14 intent samples to train on. Per intent there is information on:
- "tag", a shorthand for the topic of the conversation,
- "patterns", the user input, multiple alternatives possible for this topic,
- "responses", the chatbot replies,
- "context", a field that correlates an entry with the tag field of another entry, for multiple interactions with user

## Load json into lists

In [3]:
words = []                # all words in user input pattern
documents = []            # tokenized words and tags
classes = []              # tags
ignore_words = ['?', '!'] # also called stop words

data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # tokenize each word in user pattern and add to words list
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        # add pattern token and tag to documents list
        documents.append((w, intent['tag']))

        # add tag to classes list (only unique)
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

## Process lists

In [4]:
lemmatizer = WordNetLemmatizer()

# lemmaztize and lower each item in words list
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

# convert to set to remove duplicates and back to list
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination of user input (patterns) and tags (intents)
print("{} documents {}".format(len(documents),documents))

# classes = unique intents
print("\n{} classes {}".format(len(classes),classes))

# words = all words, vocabulary
print("\n{} unique lemmatized words {}".format(len(words),words))

47 documents [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'),

In [5]:
# write words and classes to pickle files
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

## Preprocessing: Create Feature (X) and Labels (Y) Training Set

In [6]:
# create training list
training = []

# create empty array for output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create bag of words array with 1, if word match found in current pattern
    for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
# shuffle features, make np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


## Create DNN

In [7]:
# Output neuron predicts intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [8]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fit model
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# save weights to file
model.save('chatbot_model.h5', history)

print("model created")

Epoch 1/200
47/47 [==============================] - 0s 7ms/step - loss: 2.2656 - accuracy: 0.1489
Epoch 2/200
47/47 [==============================] - 0s 638us/step - loss: 2.1533 - accuracy: 0.0851
Epoch 3/200
47/47 [==============================] - 0s 636us/step - loss: 2.1048 - accuracy: 0.2766
Epoch 4/200
47/47 [==============================] - 0s 847us/step - loss: 2.0223 - accuracy: 0.3191
Epoch 5/200
47/47 [==============================] - 0s 640us/step - loss: 1.9430 - accuracy: 0.3617
Epoch 6/200
47/47 [==============================] - 0s 639us/step - loss: 1.8928 - accuracy: 0.3191
Epoch 7/200
47/47 [==============================] - 0s 645us/step - loss: 1.6922 - accuracy: 0.4255
Epoch 8/200
47/47 [==============================] - 0s 844us/step - loss: 1.5418 - accuracy: 0.4894
Epoch 9/200
47/47 [==============================] - 0s 639us/step - loss: 1.4640 - accuracy: 0.6170
Epoch 10/200
47/47 [==============================] - 0s 641us/step - loss: 1.3000 - accuracy